In [2]:
import Pkg
Pkg.add(url="https://github.com/lucabrugnolini/VectorAutoregressions.jl")

     Cloning git-repo `https://github.com/lucabrugnolini/VectorAutoregressions.jl`
    Updating git-repo `https://github.com/lucabrugnolini/VectorAutoregressions.jl`
    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed EllipsisNotation ─ v0.4.0
   Installed GrowableArrays ─── v0.1.0
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [dd6ac5f8] + VectorAutoregressions v0.1.0 `https://github.com/lucabrugnolini/VectorAutoregressions.jl#master`
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
⌅ [da5c29d0] + EllipsisNotation v0.4.0
  [7ebbc34c] + GrowableArrays v0.1.0
  [dd6ac5f8] + VectorAutoregressions v0.1.0 `https://github.com/lucabrugnolini/VectorAutoregressions.jl#master`
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ EllipsisNotation
  ✓ 

In [1]:
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using VectorAutoregressions
using CSV
using Test
include("utils.jl")

print_percentiles (generic function with 1 method)

In [2]:
columns = [:Date, :short_us, :real_us, :term_us]
data_df = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Macro_Data"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
macro_data = TimeArray(data_df; timestamp = :Date)

209×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1972-01-01 to 2024-01-01
┌────────────┬────────────┬──────────────┬─────────────┐
│            │ short_us   │ real_us      │ term_us     │
├────────────┼────────────┼──────────────┼─────────────┤
│ 1972-01-01 │     0.0388 │    0.0023571 │   0.0215333 │
│ 1972-04-01 │     0.0455 │   0.00326571 │   0.0159333 │
│ 1972-07-01 │  0.0493333 │   0.00268831 │   0.0135667 │
│ 1972-10-01 │      0.053 │   0.00368448 │   0.0107333 │
│ 1973-01-01 │      0.064 │   0.00337313 │  0.00203333 │
│ 1973-04-01 │  0.0767333 │  -0.00326431 │ -0.00866667 │
│ 1973-07-01 │   0.103533 │   0.00377342 │  -0.0314667 │
│ 1973-10-01 │  0.0923667 │   5.89786e-5 │  -0.0248333 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │
│ 2022-07-01 │  0.0282333 │  -0.00602734 │  0.00283333 │
│ 2022-10-01 │  0.0427333 │   0.00696215 │ -0.00443333 │
│ 2023-01-01 │  0.0475333 │   0.00141827 │  -0.0110667 │
│ 2023-04-01 │  0.0513333 │  0.000898898 │     -0.0154 │
│ 2023-07-01

In [3]:
data_df = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[1:end, [:Date, :USA_Net]]
dropmissing!(data_df, disallowmissing=true)
data = TimeArray(data_df; timestamp = :Date)
data = collapse(data, Dates.quarter, last)

dates = timestamp(data) .+ Day(1)

data = TimeArray(data; timestamp = dates)

equity = percentchange(data, :log)

205×1 TimeArray{Float64, 1, Date, Vector{Float64}} 1973-04-01 to 2024-04-01
┌────────────┬────────────┐
│            │ USA_Net    │
├────────────┼────────────┤
│ 1973-04-01 │ -0.0833335 │
│ 1973-07-01 │ -0.0532413 │
│ 1973-10-01 │  0.0402782 │
│ 1974-01-01 │ -0.0900348 │
│ 1974-04-01 │ -0.0428685 │
│ 1974-07-01 │ -0.0874197 │
│ 1974-10-01 │  -0.301121 │
│ 1975-01-01 │  0.0939096 │
│     ⋮      │     ⋮      │
│ 2022-10-01 │ -0.0493364 │
│ 2023-01-01 │  0.0674734 │
│ 2023-04-01 │  0.0731963 │
│ 2023-07-01 │  0.0824719 │
│ 2023-10-01 │ -0.0322941 │
│ 2024-01-01 │   0.111622 │
│ 2024-04-01 │  0.0979943 │
└────────────┴────────────┘
           190 rows omitted

In [4]:
mean(equity) .* 4

1×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2024-04-01 to 2024-04-01
┌────────────┬───────────┐
│            │ USA_Net   │
├────────────┼───────────┤
│ 2024-04-01 │ 0.0917981 │
└────────────┴───────────┘

In [5]:
e_premium = equity .- macro_data[:short_us] ./ 4

204×1 TimeArray{Float64, 1, Date, Vector{Float64}} 1973-04-01 to 2024-01-01
┌────────────┬──────────────────┐
│            │ USA_Net_short_us │
├────────────┼──────────────────┤
│ 1973-04-01 │        -0.102517 │
│ 1973-07-01 │       -0.0791246 │
│ 1973-10-01 │        0.0171865 │
│ 1974-01-01 │        -0.111635 │
│ 1974-04-01 │       -0.0702268 │
│ 1974-07-01 │        -0.117486 │
│ 1974-10-01 │        -0.324538 │
│ 1975-01-01 │        0.0771846 │
│     ⋮      │        ⋮         │
│ 2022-07-01 │         -0.19188 │
│ 2022-10-01 │       -0.0600197 │
│ 2023-01-01 │          0.05559 │
│ 2023-04-01 │         0.060363 │
│ 2023-07-01 │        0.0689052 │
│ 2023-10-01 │       -0.0457858 │
│ 2024-01-01 │        0.0984798 │
└────────────┴──────────────────┘
                 189 rows omitted

In [6]:
mean(e_premium) .* 4

1×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2024-01-01 to 2024-01-01
┌────────────┬──────────────────┐
│            │ USA_Net_short_us │
├────────────┼──────────────────┤
│ 2024-01-01 │        0.0403383 │
└────────────┴──────────────────┘

In [7]:
returns = merge(macro_data, e_premium)

204×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1973-04-01 to 2024-01-01
┌────────────┬────────────┬──────────────┬─────────────┬──────────────────┐
│            │ short_us   │ real_us      │ term_us     │ USA_Net_short_us │
├────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ 1973-04-01 │  0.0767333 │  -0.00326431 │ -0.00866667 │        -0.102517 │
│ 1973-07-01 │   0.103533 │   0.00377342 │  -0.0314667 │       -0.0791246 │
│ 1973-10-01 │  0.0923667 │   5.89786e-5 │  -0.0248333 │        0.0171865 │
│ 1974-01-01 │     0.0864 │  -0.00655872 │  -0.0158667 │        -0.111635 │
│ 1974-04-01 │   0.109433 │ -0.000866128 │      -0.034 │       -0.0702268 │
│ 1974-07-01 │   0.120267 │ -0.000148683 │  -0.0406333 │        -0.117486 │
│ 1974-10-01 │  0.0936667 │  -0.00641213 │  -0.0169667 │        -0.324538 │
│ 1975-01-01 │     0.0669 │  -0.00137404 │      0.0085 │        0.0771846 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │        ⋮         │
│ 2022-07-01

In [8]:
returns_summarystats(returns,1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
short_us,0.0388218,0.1752,0.0499883,0.0701083,0.001,0.732581,0.0145833,0.970067,0.306504,1.28764,0.0520833
real_us,0.00869245,0.0357101,0.00267766,0.00830319,-0.0258859,0.0123656,-0.00245872,0.642757,1.23567,0.308044,0.00268779
term_us,0.0156434,0.0351,0.00984224,0.021925,-0.0406333,-0.802433,0.000825,0.882008,0.499251,0.629163,0.0119833
USA_Net_short_us,0.0856362,0.195054,0.0100846,0.0620761,-0.324538,-0.868604,-0.0312701,0.0713791,1.39014,0.117761,0.0231837


In [9]:
CSV.write("usa.csv",returns)

"usa.csv"

In [11]:
model = VAR(values(returns), 1, true)

VAR([0.0767333333333333 -0.0032643068090423034 -0.0086666666666666 -0.10251684479395795; 0.10353333333333299 0.003773416161491721 -0.031466666666666296 -0.07912463252540157; … ; 0.0539666666666666 0.012356876445824887 -0.009566666666666598 -0.045785771432493155; 0.052566666666666595 0.002684271890795644 -0.010966666666666597 0.09847984128758931], [0.10353333333333299 0.0923666666666666 … 0.0539666666666666 0.052566666666666595; 0.003773416161491721 5.897855263142099e-5 … 0.012356876445824887 0.002684271890795644; -0.031466666666666296 -0.02483333333333329 … -0.009566666666666598 -0.010966666666666597; -0.07912463252540157 0.017186488034891395 … -0.045785771432493155 0.09847984128758931], [1.0 1.0 … 1.0 1.0; 0.0767333333333333 0.10353333333333299 … 0.054266666666666595 0.0539666666666666; … ; -0.0086666666666666 -0.031466666666666296 … -0.012766666666666593 -0.009566666666666598; -0.10251684479395795 -0.07912463252540157 … 0.06890518427257061 -0.045785771432493155], [-0.0010656671197905

In [12]:
round.(model.β, digits=6)

4×5 Matrix{Float64}:
 -0.001066   1.01681   -0.229077  0.057198   0.015635
 -0.004058   0.09257    0.450456  0.090799   0.003039
  0.001739  -0.01772    0.128329  0.889771  -0.011675
  0.015426  -0.283758   0.77338   0.702965   0.034438

In [13]:
round.(model.Σ, digits=4)

4×4 Matrix{Float64}:
  0.0001   0.0  -0.0001   0.0001
  0.0      0.0  -0.0     -0.0
 -0.0001  -0.0   0.0001  -0.0
  0.0001  -0.0  -0.0      0.0071

In [14]:
A = [1 2; 3 4]


2×2 Matrix{Int64}:
 1  2
 3  4

In [15]:
B = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [16]:
A * B

2×2 Matrix{Int64}:
  7  10
 15  22

In [17]:
dot(A,B)

30

In [18]:
kron(A,B)

4×4 Matrix{Int64}:
 1   2   2   4
 3   4   6   8
 3   6   4   8
 9  12  12  16

- [ ] Normal Wishart Prior gibs sampler
- [ ] draw posterior parameters
- [ ] describe posterior parameters
- [ ] simulate
- [ ] plot simulation
- [ ] estimate momoments estimation
- [ ] calculate waic i inne miary

In [53]:
test_data = values(returns)[134:end,:]

71×4 Matrix{Float64}:
 0.0539333    0.00780662   -0.00496667  -0.0306939
 0.0532333    0.0217775    -0.00693333   0.0376527
 0.0531       0.00304751   -0.0063       0.0505816
 0.0531667   -0.00578768   -0.0047      -0.00438775
 0.0542333    0.0107117    -0.00693333   0.0446743
 0.0502333    0.00539319   -0.00763333   0.00694396
 0.0323      -0.00326106    0.00433333  -0.0417236
 0.0275667   -0.0148479     0.0113      -0.10631
 0.0305667   -0.00395082    0.00806667  -0.0300029
 0.0281667    0.0357101     0.00436667  -0.102761
 ⋮                                      
 0.00443333  -0.0214703     0.0149667    0.0939515
 0.0137      -0.0258859     0.0156      -0.05795
 0.0282333   -0.00602734    0.00283333  -0.19188
 0.0427333    0.00696215   -0.00443333  -0.0600197
 0.0475333    0.00141827   -0.0110667    0.05559
 0.0513333    0.000898898  -0.0154       0.060363
 0.0542667    0.00484838   -0.0127667    0.0689052
 0.0539667    0.0123569    -0.00956667  -0.0457858
 0.0525667    0.00268427   

In [54]:
model = VAR(test_data, 1, true)
C_OLS = round.(model.β, digits=4)

4×5 Matrix{Float64}:
 -0.0013   1.0381  -0.2897   0.0217  -0.0082
 -0.0083   0.292    0.2416   0.1802  -0.0011
  0.0035  -0.1104   0.2116   0.8725   0.0052
  0.0441  -1.1338  -0.2025  -0.7741   0.0347

## Normal Wishart Prior gibs sampler
- [x] draw covariance matrix
- [] draw coeff - zrobić, aby działało
- refactor, to ols model, i NormalWishartBVAR
- zrobić testy

In [55]:
include("financial_bvar_temp.jl")

model = FinancialBVAR.NormalWishartBVAR(test_data)

round.(transpose(model.C_OLS), digits=4)

model.C_OLS

5×4 Matrix{Float64}:
 -0.0013331   -0.00825192   0.00345063   0.0441006
  1.03807      0.29205     -0.110432    -1.13381
 -0.289741     0.241632     0.21155     -0.202543
  0.0216543    0.180209     0.872462    -0.774091
 -0.00815769  -0.00112187   0.00523774   0.0346563

In [56]:
FinancialBVAR.sample_posterior!(model)
model.Σ

PosDefException: PosDefException: matrix is not Hermitian; Cholesky factorization failed.

In [57]:
m = vec(model.C_OLS)

20-element Vector{Float64}:
 -0.0013331020531651863
  1.0380747685218707
 -0.28974104935929673
  0.021654290335881576
 -0.008157688735980348
 -0.008251915119291933
  0.2920497124096815
  0.2416324404680302
  0.18020944931212102
 -0.0011218692583331528
  0.003450631518553828
 -0.11043182655442896
  0.2115500341078264
  0.8724616374674453
  0.00523773770710714
  0.04410062611315093
 -1.1338054794291648
 -0.20254295639474978
 -0.7740911987533649
  0.03465628762211287

In [58]:
using LinearAlgebra
p = kron(model.Σ[1],inv(transpose(model.X) * model.X))

20×20 Matrix{Float64}:
  3.15224e-6   -8.93826e-5    1.63954e-5   …  -0.000464956  -3.14275e-6
 -8.93826e-5    0.00305215   -0.000776874      0.0129613     5.5649e-5
  1.63954e-5   -0.000776874   0.00415647       0.000936783   0.000332107
 -0.000125409   0.00349594    0.000252671      0.0230755     4.19658e-5
 -8.47666e-7    1.50097e-5    8.95766e-5       4.19658e-5    0.00014877
 -1.94487e-7    5.51472e-6   -1.01157e-6   …   0.00306393    2.07098e-5
  5.51472e-6   -0.000188312   4.79316e-5      -0.0854113    -0.000366711
 -1.01157e-6    4.79316e-5   -0.000256446     -0.00617313   -0.00218849
  7.73746e-6   -0.000215693  -1.55893e-5      -0.152061     -0.000276542
  5.22993e-8   -9.26071e-7   -5.52669e-6      -0.000276542  -0.000980354
 -1.72896e-6    4.90253e-5   -8.99271e-6   …  -0.000482188  -3.25922e-6
  4.90253e-5   -0.00167407    0.000426106      0.0134417     5.77114e-5
 -8.99271e-6    0.000426106  -0.00227978       0.000971501   0.000344416
  6.87851e-5   -0.00191748   -0.00013

In [59]:
MvNormal(m,p)

PosDefException: PosDefException: matrix is not Hermitian; Cholesky factorization failed.

In [115]:
X = [1 2; 3 4; 5 6]

Z = [3 1.5; 7 3.5; 11 5.5]

transpose(X) * Z


2×2 Matrix{Float64}:
  79.0  39.5
 100.0  50.0

In [141]:
1 ./ X[1:2,:]

2×2 Matrix{Float64}:
 1.0       0.5
 0.333333  0.25

In [139]:
inv(X[1:2,:])

2×2 Matrix{Float64}:
 -2.0   1.0
  1.5  -0.5